In [ ]:
pip install nibabel torch torchvision monai numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.5 MB/s eta 0:00:00


In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pathlib import Path

In [8]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

import nibabel as nib

data_directory = r"G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti"
paths = []
# for current_dir, inner_dirs, files in os.walk(data_directory):
#     for file in files:
#         if file.endswith(".nii"):
#             file_path = current_dir + "/" + file
#             paths.append(file_path.replace("\\","/"))

for root, dirs, files in os.walk(data_directory):
    for f in files:
        if f.endswith(".nii") or f.endswith(".nii.gz"):
            paths.append(os.path.join(root, f).replace("\\", "/"))

print(f"Found {len(paths)} NIfTI files.")
print("First 5 files:", paths[:5])

Found 22 NIfTI files.
First 5 files: ['G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti/UCSF-PDGM-0135_DTI_eddy_L3.nii.gz', 'G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti/UCSF-PDGM-0135_DTI_eddy_MD.nii.gz', 'G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti/UCSF-PDGM-0135_ADC.nii.gz', 'G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti/UCSF-PDGM-0135_DTI_eddy_FA.nii.gz', 'G:/Shared drives/Fall2025BrainTumorDiagnosis/UCSF-PDGM-0135_nifti/UCSF-PDGM-0135_DTI_eddy_L1.nii.gz']


In [11]:
base_path = r"G:/Shared drives/Fall2025BrainTumorDiagnosis/"

In [13]:
print("📂 Contents of Shared Drive:")
for item in sorted(os.listdir(base_path)):
    full_path = os.path.join(base_path, item)
    if os.path.isfile(full_path):
        print(f"  📄 {item}")
    else:
        print(f"  📁 {item}/")

# Get all patient folders
patient_folders = sorted([f for f in os.listdir(base_path)
                         if os.path.isdir(os.path.join(base_path, f))])

print(f"Total patient folders: {len(patient_folders)}")
print(f"\nFirst 5 patients:")
for folder in patient_folders[:5]:
    print(f"  📁 {folder}")

# Check scans in the first patient
first_patient = patient_folders[0]
patient_path = os.path.join(base_path, first_patient)
scans = sorted(os.listdir(patient_path))

print(f"\n📂 Scans in {first_patient}:")
print(f"Total scans: {len(scans)}\n")
for i, scan in enumerate(scans, 1):
    print(f"{i}. {scan}")

# Check for label files in base directory
print("\n📄 Looking for label files in base directory:")
for item in os.listdir(base_path):
    if item.endswith(('.csv', '.txt', '.xlsx', '.json')):
        print(f"  Found: {item}")

📂 Contents of Shared Drive:
  📁 UCSF-PDGM-0135_nifti/
  📁 UCSF-PDGM-0136_nifti/
  📁 UCSF-PDGM-0137_nifti/
  📁 UCSF-PDGM-0139_nifti/
  📁 UCSF-PDGM-0140_nifti/
  📁 UCSF-PDGM-0141_nifti/
  📁 UCSF-PDGM-0142_nifti/
  📁 UCSF-PDGM-0143_nifti/
  📁 UCSF-PDGM-0144_nifti/
  📁 UCSF-PDGM-0145_nifti/
  📁 UCSF-PDGM-0146_nifti/
  📁 UCSF-PDGM-0147_nifti/
  📁 UCSF-PDGM-0148_nifti/
  📁 UCSF-PDGM-0149_nifti/
  📁 UCSF-PDGM-0150_nifti/
  📁 UCSF-PDGM-0151_nifti/
  📁 UCSF-PDGM-0152_nifti/
  📁 UCSF-PDGM-0153_nifti/
  📁 UCSF-PDGM-0154_nifti/
  📁 UCSF-PDGM-0155_nifti/
  📁 UCSF-PDGM-0156_nifti/
  📁 UCSF-PDGM-0157_nifti/
  📁 UCSF-PDGM-0158_nifti/
  📁 UCSF-PDGM-0159_nifti/
  📁 UCSF-PDGM-0160_nifti/
  📁 UCSF-PDGM-0161_nifti/
  📁 UCSF-PDGM-0162_nifti/
  📁 UCSF-PDGM-0163_nifti/
  📁 UCSF-PDGM-0164_nifti/
  📁 UCSF-PDGM-0165_nifti/
  📁 UCSF-PDGM-0166_nifti/
  📁 UCSF-PDGM-0167_nifti/
  📁 UCSF-PDGM-0168_nifti/
  📁 UCSF-PDGM-0169_nifti/
  📁 UCSF-PDGM-0170_nifti/
  📁 UCSF-PDGM-0172_nifti/
  📁 UCSF-PDGM-0173_nifti/
  📁 UCSF-P

In [18]:
patient_folders = sorted([f for f in os.listdir(base_path)
                         if os.path.isdir(os.path.join(base_path, f))])

tumor_count = 0
no_tumor_count = 0
error_count = 0

print("Checking all 228 patients...\n")

for patient in patient_folders:
    tumor_seg_path = os.path.join(base_path, patient,
                                   f"{patient.replace('_nifti', '')}_tumor_segmentation.nii.gz")

    try:
        img = nib.load(tumor_seg_path).get_fdata()
        if np.any(img > 0):
            tumor_count += 1
        else:
            no_tumor_count += 1
            print(f"NO TUMOR: {patient}")  # Print patients without tumor
    except:
        error_count += 1
        print(f"ERROR loading: {patient}")

print(f"\n{'='*50}")
print(f"SUMMARY:")
print(f"  Total patients: {len(patient_folders)}")
print(f"  Has tumor: {tumor_count}")
print(f"  No tumor: {no_tumor_count}")
print(f"  Errors: {error_count}")
print(f"{'='*50}")

Checking all 228 patients...


SUMMARY:
  Total patients: 228
  Has tumor: 228
  No tumor: 0
  Errors: 0


In [15]:
csv_path="fall-2025-brain-tumor-diagnosis/Clinical-data/UCSF-PDGM-metadata_v5.csv"
df = pd.read_csv(csv_path)

# Display basic info
print(f"CSV loaded successfully!")
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")

# Show column names
print("Column names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

# Show first few rows
print("\nFirst 5 rows:")
print(df.head())

# Check if there's a patient ID column that matches our folder names
print("\nChecking for patient ID column...")
potential_id_cols = [col for col in df.columns if 'id' in col.lower() or 'patient' in col.lower() or 'case' in col.lower()]
if potential_id_cols:
    print(f"Potential ID columns: {potential_id_cols}")
    print(f"\nSample values from '{potential_id_cols[0]}':")
    print(df[potential_id_cols[0]].head(10))

CSV loaded successfully!
Shape: 501 rows × 16 columns

Column names:
  1. ID
  2. Sex
  3. Age at MRI
  4. WHO CNS Grade
  5. Final pathologic diagnosis (WHO 2021)
  6. MGMT status
  7. MGMT index
  8. 1p/19q
  9. IDH
  10. 1-dead 0-alive
  11. OS
  12. EOR
  13. Biopsy prior to imaging
  14. BraTS21 ID
  15. BraTS21 Segmentation Cohort
  16. BraTS21 MGMT Cohort

First 5 rows:
              ID Sex  Age at MRI  WHO CNS Grade  \
0  UCSF-PDGM-004   M          66              4   
1  UCSF-PDGM-005   F          80              4   
2  UCSF-PDGM-007   M          70              4   
3  UCSF-PDGM-008   M          70              4   
4  UCSF-PDGM-009   F          68              4   

  Final pathologic diagnosis (WHO 2021)    MGMT status MGMT index   1p/19q  \
0            Glioblastoma, IDH-wildtype       negative          0  unknown   
1            Glioblastoma, IDH-wildtype  indeterminate    unknown  unknown   
2            Glioblastoma, IDH-wildtype  indeterminate    unknown  unknown   
3

In [16]:
# Get patient folders
patient_folders = sorted([f for f in os.listdir(base_path)
                         if os.path.isdir(os.path.join(base_path, f))])

# Extract patient number and pad to 4 digits
df['patient_num'] = df['ID'].str.extract(r'UCSF-PDGM-(\d+)')[0]
df['folder_name'] = 'UCSF-PDGM-' + df['patient_num'].str.zfill(4) + '_nifti'

# Filter to only patients we have imaging data for
df_with_imaging = df[df['folder_name'].isin(patient_folders)]

print(f"Total patients in CSV: {len(df)}")
print(f"Total folders: {len(patient_folders)}")
print(f"Matched patients (have both data and labels): {len(df_with_imaging)}")

# Check WHO Grade distribution
print(f"\n{'='*50}")
print(f"WHO Grade distribution:")
print(df_with_imaging['WHO CNS Grade'].value_counts().sort_index())
print(f"{'='*50}")

# Show first few matches
print(f"\nFirst 5 matched patients:")
print(df_with_imaging[['ID', 'folder_name', 'WHO CNS Grade']].head())

# Create labels dictionary
labels_dict = dict(zip(df_with_imaging['folder_name'],
                       df_with_imaging['WHO CNS Grade']))

print(f"\nLabels dictionary created!")
print(f"Example entries:")
for i, (folder, grade) in enumerate(list(labels_dict.items())[:5]):
    print(f"  {folder}: Grade {grade}")

Total patients in CSV: 501
Total folders: 228
Matched patients (have both data and labels): 228

WHO Grade distribution:
WHO CNS Grade
2     39
3     24
4    165
Name: count, dtype: int64

First 5 matched patients:
                ID           folder_name  WHO CNS Grade
117  UCSF-PDGM-135  UCSF-PDGM-0135_nifti              4
118  UCSF-PDGM-136  UCSF-PDGM-0136_nifti              4
119  UCSF-PDGM-137  UCSF-PDGM-0137_nifti              4
121  UCSF-PDGM-139  UCSF-PDGM-0139_nifti              4
122  UCSF-PDGM-140  UCSF-PDGM-0140_nifti              4

Labels dictionary created!
Example entries:
  UCSF-PDGM-0135_nifti: Grade 4
  UCSF-PDGM-0136_nifti: Grade 4
  UCSF-PDGM-0137_nifti: Grade 4
  UCSF-PDGM-0139_nifti: Grade 4
  UCSF-PDGM-0140_nifti: Grade 4


In [4]:
# =============================================
# 1. LOAD AND PREPARE LABELS
# =============================================

def load_labels(csv_path, base_path):
    """
    Load labels from CSV and match to folder names

    Args:
        csv_path: Path to metadata CSV file
        base_path: Path to folder containing patient folders

    Returns:
        labels_dict: Dictionary mapping folder_name -> grade (0, 1, 2)
        patient_folders: List of patient folder names
    """
    # Load CSV
    df = pd.read_csv(csv_path)

    # Get all patient folders
    patient_folders = sorted([f for f in os.listdir(base_path)
                             if os.path.isdir(os.path.join(base_path, f))])

    # Match CSV IDs to folder names (pad numbers to 4 digits)
    df['patient_num'] = df['ID'].str.extract(r'UCSF-PDGM-(\d+)')[0]
    df['folder_name'] = 'UCSF-PDGM-' + df['patient_num'].str.zfill(4) + '_nifti'

    # Filter to patients with imaging data
    df_matched = df[df['folder_name'].isin(patient_folders)]

    # Convert WHO grades to class indices: Grade 2->0, Grade 3->1, Grade 4->2
    grade_to_class = {2: 0, 3: 1, 4: 2}
    df_matched['class_label'] = df_matched['WHO CNS Grade'].map(grade_to_class)

    # Create labels dictionary
    labels_dict = dict(zip(df_matched['folder_name'], df_matched['class_label']))

    print(f"Loaded {len(labels_dict)} patients with labels")
    print(f"Class distribution: {df_matched['WHO CNS Grade'].value_counts().sort_index()}")

    return labels_dict, patient_folders


# =============================================
# 2. DATASET CLASS
# =============================================

class MRIDataset(Dataset):
    def __init__(self, base_path, patient_list, labels_dict, slice_idx=120, selected_modalities=None):
        """
        Dataset for multi-channel 2D MRI slices

        Args:
            base_path: Path to folder containing patient folders
            patient_list: List of patient folder names to include
            labels_dict: Dictionary mapping folder_name -> label
            slice_idx: Which slice to extract from 3D volume (default: 120)
            selected_modalities: List of modality names to use. If None, uses all available.
        """
        self.base_path = base_path
        self.patient_list = patient_list
        self.labels_dict = labels_dict
        self.slice_idx = slice_idx

        # Define modality order (consistent for all patients)
        # Excluding non-NIfTI files and segmentation masks
        self.all_modalities = [
            'ADC', 'FLAIR', 'T1', 'T1c', 'T2', "brain_segmentation", "tumor_segmentation"
        ]

        # Use selected modalities or all
        self.modalities = selected_modalities if selected_modalities else self.all_modalities

        print(f"Using {len(self.modalities)} modalities: {self.modalities[:3]}... (showing first 3)")

    def __len__(self):
        return len(self.patient_list)

    def __getitem__(self, idx):
        patient_folder = self.patient_list[idx]
        patient_id = patient_folder.replace('_nifti', '')
        patient_path = Path(self.base_path) / patient_folder  # ✅ Pathlib join

        channels = []
        for modality in self.modalities:
            file_path = patient_path / f"{patient_id}_{modality}.nii.gz"  # ✅ Safe path

            try:
                # Load 3D volume
                img_3d = nib.load(str(file_path)).get_fdata().astype(np.float32)  # ✅ Convert to string

                # Extract 2D slice
                if img_3d.shape[2] > self.slice_idx:
                    img_2d = img_3d[:, :, self.slice_idx]
                else:
                    img_2d = img_3d[:, :, img_3d.shape[2] // 2]

                # Normalize to [0, 1]
                img_min, img_max = img_2d.min(), img_2d.max()
                if img_max > img_min:
                    img_2d = (img_2d - img_min) / (img_max - img_min)
                else:
                    img_2d = np.zeros_like(img_2d)

                channels.append(img_2d)

            except Exception as e:
                # If file missing or error, use zero-filled channel
                print(f"Warning: Could not load {modality} for {patient_folder}: {e}")
                channels.append(np.zeros((240, 240), dtype=np.float32))

        # Stack channels: (num_channels, H, W)
        img_tensor = np.stack(channels, axis=0)
        label = self.labels_dict[patient_folder]

         # Get label
        label = self.labels_dict[patient_folder]

        return torch.tensor(img_tensor, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# =============================================
# 3. CNN MODEL
# =============================================

class CNN2D(nn.Module):
    def __init__(self, n_channels=7, n_classes=3):
        """
        2D CNN for multi-channel MRI classification

        Args:
            n_channels: Number of input channels (MRI modalities)
            n_classes: Number of output classes (3 for WHO grades 2,3,4)
        """
        super().__init__()

        self.conv_layers = nn.Sequential(
            # Block 1: n_channels -> 32
            nn.Conv2d(n_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 240x240 -> 120x120

            # Block 2: 32 -> 64
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 120x120 -> 60x60

            # Block 3: 64 -> 128
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 60x60 -> 30x30

            # Block 4: 128 -> 256
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 30x30 -> 15x15
        )

        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 15 * 15, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc_layers(x)
        return x


# =============================================
# 4. TRAINING FUNCTION
# =============================================

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5, device='cuda'):
    """
    Train the CNN model

    Returns:
        train_losses, val_losses, train_accs, val_accs
    """
    model = model.to(device)

    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100. * correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    return train_losses, val_losses, train_accs, val_accs


# =============================================
# 5. MAIN EXECUTION
# =============================================

if __name__ == "__main__":
    # Paths
    csv_path = "fall-2025-brain-tumor-diagnosis/Clinical-data/UCSF-PDGM-metadata_v5.csv"
    base_path = r"G:/Shared drives/Fall2025BrainTumorDiagnosis/"

    # Load labels
    labels_dict, all_patients = load_labels(csv_path, base_path)

    # Get patients with labels
    patients_with_labels = list(labels_dict.keys())

    # Train/val split (80/20)
    train_patients, val_patients = train_test_split(
        patients_with_labels, test_size=0.2, random_state=42,
        stratify=[labels_dict[p] for p in patients_with_labels]
    )

    print(f"\nTrain: {len(train_patients)}, Val: {len(val_patients)}")

    # Create datasets
    train_dataset = MRIDataset(base_path, train_patients, labels_dict, slice_idx=120)
    val_dataset = MRIDataset(base_path, val_patients, labels_dict, slice_idx=120)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

    # Initialize model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\nUsing device: {device}")

    model = CNN2D(n_channels=7, n_classes=3)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train
    print("\nStarting training...")
    train_losses, val_losses, train_accs, val_accs = train_model(
        model, train_loader, val_loader, criterion, optimizer,
        num_epochs=20, device=device
    )

    # Plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Val Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.set_title('Training and Validation Loss')

    ax2.plot(train_accs, label='Train Acc')
    ax2.plot(val_accs, label='Val Acc')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.set_title('Training and Validation Accuracy')

    plt.tight_layout()
    plt.show()

    print("\nTraining complete!")

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_2964\3704298045.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matched['class_label'] = df_matched['WHO CNS Grade'].map(grade_to_class)


Loaded 228 patients with labels
Class distribution: WHO CNS Grade
2     39
3     24
4    165
Name: count, dtype: int64

Train: 182, Val: 46
Using 7 modalities: ['ADC', 'FLAIR', 'T1']... (showing first 3)
Using 7 modalities: ['ADC', 'FLAIR', 'T1']... (showing first 3)

Using device: cpu

Starting training...


KeyboardInterrupt: 

In [ ]:
for g in optimizer.param_groups:
    g['lr'] = g['lr'] * 0.3
print("Reduced LR:", optimizer.param_groups[0]['lr'])

Reduced LR: 0.0003


#Save the model to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
save_path = "/content/drive/MyDrive/brain_tumor_cnn/model_final.pth"
torch.save(model.state_dict(), save_path)
print("Model saved to:", save_path)